In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages


In [53]:
select_neurons = ['Neuron_1', 'Neuron_2']

spike_inf = pd.read_csv("/media/ubuntu/sda/data/mouse5/output/analysis_grating/02_merge_new/spike_inf_refinhe.tsv", sep = '\t')
cluster_inf = pd.read_csv("/media/ubuntu/sda/data/mouse5/output/analysis_grating/02_merge_new/clusteri_inf_refinhe.tsv", sep = '\t')

In [54]:
trigger_time = pd.read_csv("/media/ubuntu/sda/data/mouse5/output/analysis_grating/01_get_trigger/trigger_time_dynamic.tsv", sep = '\t').iloc[:, 1:]

import pickle
with open('/media/ubuntu/sda/data/mouse5/output/analysis_grating/02_merge_new/outlier_dynamic.pkl', 'rb') as f:
    outlier = pickle.load(f)


In [55]:
trigger_time['start'] = trigger_time['start'] + 10000
trigger_time['expand_end'] = trigger_time['start'] + 25000
trigger_time['end'] = trigger_time['start'] + 20000
trigger_time['expand_start'] = trigger_time['start'] - 5000

In [56]:
for i in outlier:
    date = i.split("_")[0]
    image = i.split("_")[1] + "_" + i.split("_")[2]
    trigger_time = trigger_time[~((trigger_time['date'] == int(date)) & (trigger_time['order'] == image))]

In [57]:
date_order = ['030222', '042422', '052322', '062422', '092222', '112822', '122322', '012123', '022423', '032323', '042323',
              '052423', '062323', '072123']

In [58]:
filter_outlier = {}
with PdfPages("raster_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        filter_outlier[neuron] = []
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        fig, ax = plt.subplots(figsize = (12, 7.5))
        for index, row in trigger_time.iterrows():
            date = row['date']
            image_order = row['order']
            start = row['expand_start'] 
            end = row['expand_end']
            
            filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                        (neuron_df['time'] >= start) & 
                                        (neuron_df['time'] <= end)]
            
            if not filtered_spikes.empty:
                ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), marker='|', mew=1, markersize=3, ls='', color='k')
            else:
                filter_outlier[neuron].append(index)

        ax.set_yticks([])
        #ax.axvspan(1 + (5000 - 1) * 0.5, 1 + (5000 - 1), color='gray', alpha=0.3)

        ax.set_xlabel('time (s)')
        ax.set_ylabel("")
        ax.set_title(f'Raster Plot for {neuron}')

        pdf.savefig(fig)
        plt.close()

In [59]:
from collections import Counter
counter = Counter()

for key in filter_outlier.keys():
    filtered_elements = [item for item in filter_outlier[key]]
    counter.update(filtered_elements)

filtered_counter = {element: count for element, count in counter.items() if count == 2}
filtered_counter = list(filtered_counter.keys())

In [60]:
trigger_time = trigger_time.drop(filtered_counter)
trigger_time = trigger_time.reset_index(drop = True)

In [61]:
filter_outlier = {}
with PdfPages("raster_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        filter_outlier[neuron] = []
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        fig, ax = plt.subplots(figsize = (12, 7.5))
        for index, row in trigger_time.iterrows():
            date = row['date']
            image_order = row['order']
            start = row['expand_start'] 
            end = row['expand_end']
            
            filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                        (neuron_df['time'] >= start) & 
                                        (neuron_df['time'] <= end)]
            
            if not filtered_spikes.empty:
                ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), marker='|', mew=1, markersize=3, ls='', color='k')
            else:
                filter_outlier[neuron].append(index)

        ax.set_yticks([])
        #ax.axvspan(1 + (5000 - 1) * 0.5, 1 + (5000 - 1), color='gray', alpha=0.3)

        ax.set_xlabel('time (s)')
        ax.set_ylabel("")
        ax.set_title(f'Raster Plot for {neuron}')

        pdf.savefig(fig)
        plt.close()

In [62]:
with PdfPages("raster_split_image.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]


        fig, axs = plt.subplots(3, 6, figsize=(30, 15), sharex=True, sharey=True)
        fig.subplots_adjust(hspace=0.5, wspace=0.5)

        for image in range(1, 19):  
            ax = axs[(image - 1) // 6, (image - 6) % 6]  
            trigger_time_temp = trigger_time[trigger_time['image'] == image]

            for index, row in trigger_time_temp.iterrows():
                date = row['date']
                image_order = row['order']
                start = row['expand_start'] 
                end = row['expand_end'] 

                filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                            (neuron_df['time'] >= start) & 
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), 
                            marker='|', mew=1, markersize=3, ls='', color='k')


            ax.set_xticks([])
            ax.set_yticks([])

        pdf.savefig(fig)
        plt.close()

In [63]:
mean_spike_rate_dict = {}

with PdfPages("peth_lineplot_all.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        plt.figure(figsize=(4, 2.5))
        spike_rates_all = np.empty((99))

        for image in range(1, 19):
            neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

            dates = trigger_time[trigger_time['image'] == image]['date'].unique()
            date_indices = np.arange(len(dates))  

            spike_rates = []
            std_rates = []

            for date in dates:
                trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
                all_spikes = []

                for index, row in trigger_time_temp.iterrows():
                    start = row['expand_start'] 
                    end = row['expand_end']

                    filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                                (neuron_df['time'] >= start) &
                                                (neuron_df['time'] <= end)]

                    if not filtered_spikes.empty:
                        relative_spikes = filtered_spikes['time'] - start
                        all_spikes.extend(relative_spikes)

                if all_spikes:
                    time_bins = np.linspace(0, max(all_spikes), 100)
                    spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                    spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                    spike_rates.append(spike_rate)
                spike_rates_all = np.vstack((spike_rates_all, spike_rates))

        mean_spike_rate = np.mean(spike_rates_all, axis=0)
        std_spike_rate = np.std(spike_rates_all, axis=0)

        plt.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth = 2)
        plt.plot(time_bins[:-1], mean_spike_rate - std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
        plt.plot(time_bins[:-1], mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
        plt.title(neuron)
        plt.xticks([])
        plt.yticks([])
        pdf.savefig()
        plt.close()

In [ ]:
spike_rates_all = pd.DataFrame()

for image in range(1, 19):
    for neuron in select_neurons:
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates = []
        std_rates = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes = []

            for index, row in trigger_time_temp.iterrows():
                start = row['expand_start']
                end = row['expand_end']

                filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                            (neuron_df['time'] >= start) &
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    relative_spikes = filtered_spikes['time'] - start
                    all_spikes.extend(relative_spikes)

            if all_spikes:
                time_bins = np.linspace(0, max(all_spikes), 500)
                spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                spike_rates.append(spike_rate)

            spike_rates_all = pd.concat((spike_rates_all, pd.DataFrame(spike_rates)), axis= 0)

In [66]:
spike_rates_all_dict = {}

for neuron in select_neurons:
    spike_rates_all_dict[neuron] = pd.DataFrame()
    for image in range(1, 19):
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates = []
        std_rates = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes = []

            for index, row in trigger_time_temp.iterrows():
                start = row['expand_start']
                end = row['expand_end']

                filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                            (neuron_df['time'] >= start) &
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    relative_spikes = filtered_spikes['time'] - start
                    all_spikes.extend(relative_spikes)

            if all_spikes:
                time_bins = np.linspace(0, max(all_spikes), 500)
                spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                spike_rates.append(spike_rate)

            spike_rates_all_dict[neuron] = pd.concat((spike_rates_all_dict[neuron], pd.DataFrame(spike_rates)), axis= 0)

In [67]:
split_point = []
with PdfPages("peth_diff.pdf") as pdf:
    for neuron in spike_rates_all_dict.keys():
        max_x = np.argmax(spike_rates_all_dict[neuron].mean(axis = 0)[110:150]) + 110
        split_point.append(max_x)
        plt.figure()
        plt.plot(range(499), spike_rates_all_dict[neuron].mean(axis = 0), alpha=0.7, color = "#70A5D9")
        plt.scatter(max_x, spike_rates_all_dict[neuron].mean(axis = 0)[max_x], color='#F0868C',s=60)
        pdf.savefig()
        plt.close()

In [68]:
with PdfPages("peth_diff_overlay.pdf") as pdf:
    plt.figure(figsize=(10, 6))  
    
    colors = ['#70A5D9', '#F0868C', '#AFC7E8', '#F9BEB9', '#5977A0', '#D8576B', '#3B526E', '#E6919C']  
    color_index = 0  

    split_point = []
    for neuron in spike_rates_all_dict.keys():
        max_x = np.argmax(spike_rates_all_dict[neuron].mean(axis = 0)[110:150]) + 110
        split_point.append(max_x)
        
        plt.plot(range(499), spike_rates_all_dict[neuron].mean(axis=0), alpha=0.7, color=colors[color_index % len(colors)], label=f'Neuron {neuron}')
        plt.scatter(max_x, spike_rates_all_dict[neuron].mean(axis=0)[max_x], color='#F0868C', s=60)
        color_index += 1  
    
    plt.tight_layout()
    
    pdf.savefig()
    plt.close()

In [69]:
from scipy import stats

num_neurons = len(spike_inf['Neuron'].unique())
num_images = 12
result_matrix = np.zeros((num_neurons, num_images), dtype=int)
pvalue_matrix = np.ones((num_neurons, num_images))
diff_matrix = np.ones((num_neurons, num_images))

for image in range(1, 13):
    num = 0
    image_mean_spike_rate_data = []
    for neuron_idx, neuron in enumerate(spike_inf['Neuron'].unique()):

        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates_first = []
        spike_rates_second = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes_first = []
            all_spikes_second = []

            for index, row in trigger_time_temp.iterrows():
                start = row['start'] 
                end = row['expand_start'] + 30000/500 * np.array(split_point).mean()

                filtered_spikes_first = neuron_df[(neuron_df['date'] == date) &
                                                  (neuron_df['time'] >= start) &
                                                  (neuron_df['time'] <= end)]

                if not filtered_spikes_first.empty:
                    relative_spikes_first = filtered_spikes_first['time'] - start
                    all_spikes_first.extend(relative_spikes_first)

                start_2 = row['start'] - 30000/500 * np.array(split_point).mean() + 5000
                end_2 = row['start'] 

                filtered_spikes_second = neuron_df[(neuron_df['date'] == date) &
                                                   (neuron_df['time'] >= start_2) &
                                                   (neuron_df['time'] <= end_2)]

                if not filtered_spikes_second.empty:
                    relative_spikes_second = filtered_spikes_second['time'] - start_2
                    all_spikes_second.extend(relative_spikes_second)

            if all_spikes_first:
                time_bins_first = np.linspace(0, max(all_spikes_first), 100)
                spike_counts_first, _ = np.histogram(all_spikes_first, bins=time_bins_first)
                spike_rate_first = spike_counts_first / (time_bins_first[1] - time_bins_first[0])
                spike_rates_first.append(spike_rate_first)

            if all_spikes_second:
                time_bins_second = np.linspace(0, max(all_spikes_second), 100)
                spike_counts_second, _ = np.histogram(all_spikes_second, bins=time_bins_second)
                spike_rate_second = spike_counts_second / (time_bins_second[1] - time_bins_second[0])
                spike_rates_second.append(spike_rate_second)

        mean_spike_rate_first = np.mean(spike_rates_first, axis=0) if spike_rates_first else np.zeros(100)
        mean_spike_rate_second = np.mean(spike_rates_second, axis=0) if spike_rates_second else np.zeros(100)

        t_stat, p_value = stats.ttest_ind(mean_spike_rate_first, mean_spike_rate_second, nan_policy='omit')

        if p_value < 0.01:
            if np.mean(mean_spike_rate_first) > np.mean(mean_spike_rate_second):
                result_matrix[neuron_idx, image-1] = 1
                diff_matrix[neuron_idx, image-1] = np.mean(mean_spike_rate_first)/np.mean(mean_spike_rate_second)
            else:
                result_matrix[neuron_idx, image-1] = -1
                diff_matrix[neuron_idx, image-1] = np.mean(mean_spike_rate_second)/np.mean(mean_spike_rate_first)
        else:
            result_matrix[neuron_idx, image-1] = 0
            diff_matrix[neuron_idx, image-1] = 1
        pvalue_matrix[neuron_idx, image-1] = p_value
        


neurons = spike_inf['Neuron'].unique()
images = range(1, 13)
result_df_phase1 = pd.DataFrame(result_matrix, index=neurons, columns=images)
pvalue_df_phase1 = pd.DataFrame(pvalue_matrix, index=neurons, columns=images)
diff_df_phase1 = pd.DataFrame(diff_matrix, index=neurons, columns=images)

result_df_phase1.to_csv("glimpse_result_df.csv")
pvalue_df_phase1.to_csv("glimpse_pvalue_df.csv")
diff_df_phase1.to_csv("glimpse_diff_df.csv")

In [70]:
from math import pi

result_df_phase1 = result_df_phase1[[1, 7,2,8,3,9,4,10,5,11,6,12]]
pvalue_df_phase1 = pvalue_df_phase1[[1, 7,2,8,3,9,4,10,5,11,6,12]]

neurons = result_df_phase1.index
images = result_df_phase1.columns

all_pvalues = pvalue_df_phase1.values.flatten()
all_pvalues = all_pvalues[all_pvalues != 0]  
all_log_pvalues = -np.log(all_pvalues)

with PdfPages('glimpse_neuron_radar_plots.pdf') as pdf:
    for neuron in neurons:
        fig = plt.figure(figsize=(5, 4))
        ax = fig.add_subplot(111, polar=True)
        
        results = result_df_phase1.loc[neuron].values
        pvalues = pvalue_df_phase1.loc[neuron].values
        
        angles = np.linspace(0, 2*np.pi, len(images), endpoint=False).tolist()
        
        for j, (result, pvalue, image) in enumerate(zip(results, pvalues, images)):
            if pvalue == 0:
                height = 0
            else:
                height = -np.log(pvalue)
            
            if result == 1:
                color = "#F9BEB9"
            elif result == 0:
                color = 'lightgrey'
            else:
                color = '#AFC7E8'
            
            ax.bar(angles[j], height, width=2*np.pi/len(images), bottom=0.0, color=color, alpha=0.8)
                
        ax.set_xticks(angles)
        ax.set_xticklabels(images)
        ax.set_title(neuron, pad=20, x=-0.1)
        
        ax.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
        ax.tick_params(axis='y', labelsize=8)
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()

In [72]:
from scipy import stats

num_neurons = len(spike_inf['Neuron'].unique())
num_images = 12
result_matrix = np.zeros((num_neurons, num_images), dtype=int)
pvalue_matrix = np.ones((num_neurons, num_images))
diff_matrix = np.ones((num_neurons, num_images))

for image in range(1, 13):
    num = 0
    image_mean_spike_rate_data = []
    for neuron_idx, neuron in enumerate(spike_inf['Neuron'].unique()):

        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates_first = []
        spike_rates_second = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes_first = []
            all_spikes_second = []

            for index, row in trigger_time_temp.iterrows():
                start = row['start'] + 10000
                end = row['end']

                filtered_spikes_first = neuron_df[(neuron_df['date'] == date) &
                                                  (neuron_df['time'] >= start) &
                                                  (neuron_df['time'] <= end)]

                if not filtered_spikes_first.empty:
                    relative_spikes_first = filtered_spikes_first['time'] - start
                    all_spikes_first.extend(relative_spikes_first)

                start_2 = row['start']- (end - start)
                end_2 = row['start'] 

                filtered_spikes_second = neuron_df[(neuron_df['date'] == date) &
                                                   (neuron_df['time'] >= start_2) &
                                                   (neuron_df['time'] <= end_2)]

                if not filtered_spikes_second.empty:
                    relative_spikes_second = filtered_spikes_second['time'] - start_2
                    all_spikes_second.extend(relative_spikes_second)

            if all_spikes_first:
                time_bins_first = np.linspace(0, max(all_spikes_first), 500)
                spike_counts_first, _ = np.histogram(all_spikes_first, bins=time_bins_first)
                spike_rate_first = spike_counts_first / (time_bins_first[1] - time_bins_first[0])
                spike_rates_first.append(spike_rate_first)

            if all_spikes_second:
                time_bins_second = np.linspace(0, max(all_spikes_second), 500)
                spike_counts_second, _ = np.histogram(all_spikes_second, bins=time_bins_second)
                spike_rate_second = spike_counts_second / (time_bins_second[1] - time_bins_second[0])
                spike_rates_second.append(spike_rate_second)

        mean_spike_rate_first = np.mean(spike_rates_first, axis=0) if spike_rates_first else np.zeros(100)
        mean_spike_rate_second = np.mean(spike_rates_second, axis=0) if spike_rates_second else np.zeros(100)

        t_stat, p_value = stats.ttest_ind(mean_spike_rate_first, mean_spike_rate_second, nan_policy='omit')

        if p_value < 1e-3:
            if np.mean(mean_spike_rate_first) > np.mean(mean_spike_rate_second):
                result_matrix[neuron_idx, image-1] = 1
            else:
                result_matrix[neuron_idx, image-1] = -1
        else:
            result_matrix[neuron_idx, image-1] = 0
        pvalue_matrix[neuron_idx, image-1] = p_value
        diff_matrix[neuron_idx, image-1] = np.mean(mean_spike_rate_first)/np.mean(mean_spike_rate_second)


neurons = spike_inf['Neuron'].unique()
images = range(1, 13)
result_df_phase2 = pd.DataFrame(result_matrix, index=neurons, columns=images)
pvalue_df_phase2 = pd.DataFrame(pvalue_matrix, index=neurons, columns=images)
diff_df_phase2 = pd.DataFrame(diff_matrix, index=neurons, columns=images)

result_df_phase2.to_csv("last_result_df.csv")
pvalue_df_phase2.to_csv("last_pvalue_df.csv")
diff_df_phase2.to_csv("last_diff_df.csv")

In [73]:
result_df_phase2 = result_df_phase2[[1, 7,2,8,3,9,4,10,5,11,6,12]]
pvalue_df_phase2 = pvalue_df_phase2[[1, 7,2,8,3,9,4,10,5,11,6,12]]

neurons = result_df_phase2.index
images = result_df_phase2.columns


with PdfPages('last_neuron_radar_plots.pdf') as pdf:
    for neuron in neurons:
        fig = plt.figure(figsize=(5, 4))
        ax = fig.add_subplot(111, polar=True)
        
        results = result_df_phase2.loc[neuron].values
        pvalue = pvalue_df_phase2.loc[neuron].values
        pvalue = -np.log(pvalue)
        
        angles = np.linspace(0, 2*np.pi, len(images), endpoint=False).tolist()
        
        for j, (result, pvalue, image) in enumerate(zip(results, pvalue, images)):
            if pvalue == 0:
                height = 0
            else:
                height = pvalue
            
            if result == 1:
                color = "#F9BEB9"
            elif result == 0:
                color = 'lightgrey'
            else:
                color = '#AFC7E8'
            
            ax.bar(angles[j], height, width=2*np.pi/len(images), bottom=0.0, color=color)
        
        
        ax.set_xticks(angles)
        ax.set_xticklabels(images)
        ax.set_title(neuron, pad=20, x=-0.1)
        
        ax.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
        
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()

In [74]:
trigger_time.to_csv("trigger_time.csv")